In [1]:
import config #file with client_id and secret for the Spotify API
import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [26]:
list_of_playlists=["Top 100 Greatest Songs of All Time", "Pop Hits 2024", "Pop Party", "Rock Hits", "Rock Classics", "Big rock tracks from 2000", "Hip-Hop 2023", "Hip Hop Dance Playlist", "Hip-Hop Classics Party", "EDM House mix", "Eletronic Music Mix", "Electronic Dance Music", "Techno Music 2024", "Techno Remix", "Viva Latino", "Latino Dance Hits", "The 100 most streamed songs on Spotify", "RnB Hits 2024", "Jazz Classics", "Best Classical Music", "kids playlist 2023", "Hot hits Deutschland", "Happy Dance"]
len(list_of_playlists)

23

In [27]:
#def function fo extracting tracks from the playlists and place them into list

#function to get playlist_id

def playlist_id(name):
    result=sp.search(q=name,limit=5,type="playlist", market="GB")
    if result["playlists"]["items"]:
        playlist_id=result["playlists"]["items"][0]["id"]
        print(f"ID for {name} is found, ID:{playlist_id}")
        #return playlist_id
        return get_playlist_tracks ("Spotify", playlist_id)
    else:
        print("not found")

#function to get the list of tracks from specific playlist_id    
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return songs_ids(tracks)
       
#function to get a list of songs
def songs_ids(tracks):
    list_song_ids=[]
    for i in tracks:
        list_song_ids.append(i['track']['id'])
    return chunk(list_song_ids)

#function to do the chunk thing
def chunk(list_song_ids):
    chunk_size = 100
    list_chopped = [list_song_ids[i:i+chunk_size] for i in range(0,len(list_song_ids), 100)]
    return get_AF(list_chopped)

#function to get AF    
def get_AF(list_chopped):
    list_af = []
    for i in list_chopped:
        list_af.extend(sp.audio_features(i))
    df=pd.DataFrame(list_af)
    return df

In [28]:
#test all functions with the 1st playlist
todays_top_hits=playlist_id("today's top hits")

ID for today's top hits is found, ID:37i9dQZF1DXcBWIGoYBM5M


In [29]:
#creating an empty DF to fill
final_df=pd.DataFrame()

In [ ]:
#running functions for each genre from the list

for playlist in list_of_playlists:
    df_temp=playlist_id(playlist)
    final_df=pd.concat([final_df, df_temp])

ID for Top 100 Greatest Songs of All Time is found, ID:6i2Qd6OpeRBAzxfscNXeWp
ID for Pop Hits 2024 is found, ID:5TDtuKDbOhrfW7C58XnriZ
ID for Pop Party is found, ID:2WwJVcVXsJP2dkfKVrOY4h
ID for Rock Hits is found, ID:3UNOpyCZYYtjd24JuFgXLM
ID for Rock Classics is found, ID:37i9dQZF1DWXRqgorJj26U
ID for Big rock tracks from 2000 is found, ID:37i9dQZF1DX6rsDrBNGuWW
ID for Hip-Hop 2023 is found, ID:0HpT5j5D5vHL0sPyecTKQr
ID for Hip Hop Dance Playlist is found, ID:5B46dEOFzUu8z3uPC9lBLt
ID for Hip-Hop Classics Party is found, ID:37i9dQZF1DXbkfWVLd8wE3
ID for EDM House mix is found, ID:37i9dQZF1EIdDyy28MYSyS
ID for Eletronic Music Mix is found, ID:0MJjDVvC20xinTETjoZrvM
ID for Electronic Dance Music is found, ID:6JTc4cdN83XTFgsUB3XLPA
ID for Techno Music 2024 is found, ID:18vUeZ9BdtMRNV6gI8RnR6
ID for Techno Remix is found, ID:6Q6nROUKNVXTu7UNpQXmzc
ID for Viva Latino is found, ID:37i9dQZF1DX10zKzsJ2jva
ID for Latino Dance Hits is found, ID:1amPv4WLmtqRrPUSzxADyG
ID for The 100 most stream

In [ ]:
#dropping duplicates from the final DF
final_df=final_df.drop_duplicates()

In [ ]:
#ready #checking NAs
print(final_df.isna().sum()>0)

final_df

In [ ]:
final_df.to_csv('songs_with_AF.csv', index=False)